In [13]:
import sys
sys.path.insert(0, '..')
# from transformer.models import DecoderOnlyModel
from data_utils.Datasets import SerializedConcatDataset, PermSerializedConcatDataset, BinarySerializer
import pickle
import torch
import numpy as np

from transformers import AutoConfig, GPT2LMHeadModel

In [14]:
def load_model(model_id='any_tonality'):
    '''
    model_id: 'any_tonality' or 'c_major'
    '''
    with open('serializer.pkl', 'rb') as inp:
        binser = pickle.load(inp)
    
    # define model
    vocab_size = binser.vocab_size
    d_model = 256
    if model_id == 'c_major':
        num_heads = 2
        num_layers = 2
        max_seq_length = 3795
    else:
        num_heads = 4
        num_layers = 4
        max_seq_length = 1063
    d_ff = 256
    dropout = 0.3
    
    # dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    dev = torch.device("cpu")
    
    config = AutoConfig.from_pretrained(
        "gpt2",
        vocab_size=vocab_size,
        n_positions=max_seq_length,
        n_layer=num_layers,
        n_head=num_heads,
        pad_token_id=binser.padding,
        bos_token_id=binser.padding,
        eos_token_id=binser.padding,
        n_embd=d_ff
    )
    transformer = GPT2LMHeadModel(config).to(dev)
    transformer = transformer.to(dev)

    if model_id == 'c_major':
        saved_model_path = '../saved_models/melboost_cmaj_GPT2_serialized/melboost_cmaj_GPT2_serialized.pt'
    else:
        saved_model_path = '../saved_models/melboost_GPT2_serialized/melboost_GPT2_serialized.pt'
    transformer.load_state_dict(torch.load(saved_model_path), strict=False)
    
    transformer.eval()
    return transformer
# end load_model

In [15]:
def harmonize_melody_pcps_with_model_id(melody_pcps, model_id):
    model = load_model(model_id)
    binser2 = BinarySerializer()
    # melody_pcps to serialized
    x_mel = binser2.sequence_serialization( melody_pcps, np.array([]) )
    print('x_mel:', x_mel)
    # x_mel has 'end harmonizing' at the end - remove it
    x_mel = x_mel[:-1]
    # run model
    # dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    dev = torch.device("cpu")
    inp = torch.from_numpy( np.expand_dims(x_mel, axis=0)).to(dev)
    output = model(inp, attention_mask=inp != 0, output_attentions=True)
    prediction = output.logits.argmax(dim=2, keepdim=True).squeeze()
    z = prediction.cpu().numpy()
    # back to binary
    bin_all = binser2.indexes2binary( z )
    # make sure length of melody and chords are equal
    c = bin_all['chords']
    m = bin_all['melody']
    if c.shape[1] > m.shape[1]:
        c = c[:, :m.shame[1]]
    elif c.shape[1] < m.shape[1]:
        c = np.c_[c, np.zeros( (12, m.shape[1] - c.shape[1] ) )]
    return c

In [16]:
# test
# load data
npz_path = '../data/augmented_and_padded_data.npz'
data = np.load(npz_path)
melody_pcps = data['melody_pcps'].astype('float32')

# model_id = 'c_major'
model_id = 'any_tonality'

c = harmonize_melody_pcps_with_model_id(melody_pcps, model_id)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
with np.printoptions(threshold=np.inf):
    print('input: ', melody_pcps)
    print('output: ', c)